In [2]:
import random
from gekko import GEKKO
import networkx as nx
import pickle
from itertools import product
import numpy as np
import pandas as pd

In [3]:
g = nx.read_gml('networks/italy.gml', label="id")

g.edges # ezek a linkek, melyik tornybol melyik toronyba megy a link - ezek ugymond a linkek nevei

#ez lesz a H0 vektor, nem kell vektort elég ha csak egyszerüen egy szám
#Hnull = np.ones((len(g.edges))) * 6
Hnull = 6

# ezek a cut SLRG-k vagyis a link csoportok 
with open ('min_cut_SRLGs/italy', 'rb') as fp:
    all_cut_srlgs = pickle.load(fp)

cut_srlgs = [all_cut_srlgs[2]]
# ezek az intezitások, l,p,m
intensity = np.load('intensities/italy.npy')
#data[0][0]  ez az első link első poziciohoz tartozo földrendés intenzitásai


df = pd.read_csv('earthquake_probabilities/italy.csv')
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

prob_matrix = df.to_numpy()

epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

# ez ezeknek a hossza
L = len(g.edges)
P = len(epicenters)
M = len(magnitudes)
S = len(cut_srlgs)

# treschold cons 3-hoz
T = 0.01

##
cost = 1

In [4]:
a = intensity[14]
b = intensity[18]
c = intensity[26]

columns = ((a>6) & (b>6) & (c>6)).any(axis=0)
rows = ((a>6) & (b>6) & (c>6)).any(axis=1)

intensity = intensity[:,:,columns][:,rows]
prob_matrix = prob_matrix[:,columns][rows]

P,M = prob_matrix.shape

epicenters = range(P)
magnitudes = range(M)

In [ ]:
#Model
m = GEKKO()

# D. vars
deltaH = m.Array(m.Var,len(g.edges),lb=1,ub=6,integer=True)

Z = m.Array(m.Var, (S* P * M) ,lb=0,ub=1,integer=True)

Y = m.Array(m.Var, (L* P * M),lb=0,ub=1,integer=True)

print("Dec varok megvannak")

# Objective
m.Minimize(sum( cost * deltaH[l] for l,_ in enumerate(g.edges) ))

#cons1
m.Equation(sum( ((Hnull + deltaH[l]) / intensity[l,p,m]) * Y[l*P*M+p*M+m]   for l,_ in enumerate(g.edges) for p,_ in enumerate(epicenters) for m,_ in enumerate(magnitudes) ) <= 1)
print("cons1 megvan")

#cons2
for (c,s), (p,_), (m,_) in product(*[enumerate(cut_srlgs), enumerate(epicenters),enumerate(magnitudes)]):
    m.Equation(sum(Y[list(g.edges).index(linkID)*P*M+p*M+m]  for linkID in s)* Z[c*P*M+p*M+m] == len(s))
print("cons2 megvan")

#cons3
for c,s in enumerate(cut_srlgs):
    m.Equation(sum(prob_matrix[p,m] * Z[c*P*M+p*M+m] for p,_ in enumerate(epicenters) for m,_ in enumerate(magnitudes) ) < T )
print("cons3 megvan")

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

In [3]:
#### PÉLDA ####

koltseg =  [11, 3,3, 6, 1, 9, 16,78]
fejlesztes = [17,67,4,61,16,42,156, 7]

y = [1,0,1,1,0,1,1,0]
z = [0,1,1,1,0,1,1,1]

prob = [0.5, 1.2, 1.5, 1.7, 0.1, 1.8, 0.4, 1.9]

items = len(koltseg)

# Create model
m = GEKKO()

# Variables
x = m.Array(m.Var,items,lb=0,ub=1,integer=True)
fejlesztes_Dvar = m.Array(m.Var, items, lb=1,ub=6, integer=True)
b = m.Array(m.Var, items, lb=1,ub=8, integer=True)
#x2 = m.Array(m.Var, len(w),lb=0,ub=1,integer = True)

# Objective
m.Maximize(sum(koltseg[i] * fejlesztes_Dvar[i] * x[i] for i in range(items) ))

# Constraint
m.Equation(sum([(y[i] + z[i]) * x[i] for i in range(items)]) == 2)

m.Equation(sum([x[i] for i in range(items)]) == 1)

m.Equation(sum((z[i] * prob[i]) *x[i] for i in range(items)) <=1.6 )

# Optimize with APOPT
m.options.SOLVER = 1

m.solve(disp = False)

# Print the value of the variables at the optimum
print(x)
for i in range(len(x)):
    if x[i][0] == 1.0:
        print("Sorszam: {}., koltseg: {}, fejlesztes: {}".format(i,koltseg[i], fejlesztes_Dvar[i][0]))

[[0.0] [0.0] [0.0] [0.0] [0.0] [0.0] [1.0] [0.0]]
Sorszam: 6., koltseg: 16, fejlesztes: 6.0
